# **Problem Statement**

One of the challenge for all Pharmaceutical companies is to understand the persistency of drug as per the physician prescription. 


To solve this problem ABC pharma company approached an analytics company to automate this process of identification


# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import skew 
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score



# Read Dataset

In [ ]:
data_H= pd.read_csv('E:/solo projects/Data_Glacier_virtual_internship/Data_Glacier_virtual_internship/Week08/Healthcare_dataset.csv')
data_H.head() 

#  **Data Understanding**
(Preprocessing)

In [ ]:
print(f'Number of Observations: {data_H.shape[0]}')
print(f'Number of Features: {data_H.shape[1]}')

**Data Info**

In [ ]:
data_H.info()

**Features**

In [ ]:
data_H.columns


**Data types**

In [ ]:
data_H.dtypes

In [ ]:
print("Object Columns: ", data_H.select_dtypes(include = ["object"]).columns)


In [ ]:
print("Numerical Columns: ", data_H.select_dtypes(include = ["int64"]).columns)

In [ ]:
obj_col = list(data_H.select_dtypes(['object']).columns)
print(len(obj_col))
obj_col

**Size of the data**

In [ ]:
data_H.size

**Check Missing values**

In [ ]:
missing_values = data_H.isna().sum()/len(data_H)*1004
missing_values

**Unique values**

In [ ]:
data_H.nunique()

In [ ]:
data_H.describe()

In [ ]:
data_H.value_counts('Persistency_Flag')


**Check duplicates**

In [ ]:
data_H=data_H.drop_duplicates()
data_H


# No duplicate found

**Drop id column**

In [ ]:
# data_H.drop (['Ptid'], axis=1 , inplace=True)
# data_H.head()

**Outliers for numerical columns**

In [ ]:
df = data_H.select_dtypes([int, float])

fig=plt.figure(figsize=(20,20))
for i ,columns in enumerate (df,1):
    ax= plt.subplot(5,3,i)
    sns.boxplot(data= df , x=df[columns])
    ax.set_xlabel(None)
    ax.set_title(f'Distribution of {columns}')
    plt.tight_layout(w_pad=3)
plt.show()

**Check for Skewness**

In [ ]:
skew_D = df.skew().sort_values(ascending=False)
skewness = pd.DataFrame({'skew':skew_D})
skewness

skew was greater than zero so the more weight in the left tail of the distribution

**Histograms for numeric values**

In [ ]:
data_H.hist(column='Dexa_Freq_During_Rx')

In [ ]:
data_H.hist(column='Count_Of_Risks')

# Data Cleaning and Feature Engineering 

**Remove outliers by IQR**

In [ ]:
data = data_H
data


**Count_Of_Risks**

In [ ]:
maxval = data["Count_Of_Risks"].max()
print(maxval)

minval = data["Count_Of_Risks"].min()
print(minval)

# Removing Outliers from Count_Of_Risks using IQR
Q1 = data["Count_Of_Risks"].quantile(0.25)
Q3 = data["Count_Of_Risks"].quantile(0.75)
IQR = Q3-Q1
lowqe_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(lowqe_bound, upper_bound)


data = data[~(
    (data["Count_Of_Risks"] < lowqe_bound) | (data["Count_Of_Risks"] > upper_bound))]
data.shape

**Dexa_Freq_During_Rx**

In [ ]:
maxval = data["Dexa_Freq_During_Rx"].max()
print(maxval)

minval = data["Dexa_Freq_During_Rx"].min()
print(minval)

# Removing Outliers from Dexa_Freq_During_Rx using IQR
Q1 = data["Dexa_Freq_During_Rx"].quantile(0.25)
Q3 = data["Dexa_Freq_During_Rx"].quantile(0.75)
IQR = Q3-Q1
lowqe_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(lowqe_bound, upper_bound)


data = data[~(
    (data["Dexa_Freq_During_Rx"] < lowqe_bound) | (data["Dexa_Freq_During_Rx"] > upper_bound))]
data.shape

plot after remove outliers

In [ ]:
df = data.select_dtypes([int, float])

fig=plt.figure(figsize=(15,15))
for i ,columns in enumerate (df,1):
    ax= plt.subplot(5,3,i)
    sns.boxplot(data= df , x=df[columns])
    ax.set_xlabel(None)
    ax.set_title(f'Distribution of {columns}')
    plt.tight_layout(w_pad=3)
plt.show()

**Remove outliers by Z-Score**

In [ ]:
data_outliers = data_H.select_dtypes([int, float])

In [ ]:
from scipy import stats
df2 = data_outliers[(np.abs(stats.zscore(data_outliers)) < 2).all(axis=1)]

In [ ]:
df2

In [ ]:
df2.describe()

Will use the data that done by IQR to continue the next steps

Fix Skewness

In [ ]:
skew_D = data.select_dtypes([int, float]).skew().sort_values(ascending=False)
skewness = pd.DataFrame({'skew':skew_D})
skewness

In [ ]:
pt=PowerTransformer(method='yeo-johnson') 
X_power=pt.fit_transform(data.select_dtypes([int, float]))
df=pd.DataFrame(X_power,columns=data.select_dtypes([int, float]).columns)

In [ ]:
data.hist(column='Count_Of_Risks')

In [ ]:
data.hist(column='Dexa_Freq_During_Rx')

# EDA

Analysis 

**Non-Persistent have higher records in dataset**

In [ ]:
sns.countplot(x="Persistency_Flag",data=data, dodge=True)

**The age_Bucker >75 in Persistent and non-Persistent have higher values**

In [ ]:
sns.countplot(x="Persistency_Flag", hue='Age_Bucket', data=data)


In [ ]:
sns.countplot(x="Persistency_Flag", hue='Count_Of_Risks', data=data)


**The Caucasian Race in both persistent and non-Persistent have highest count**

In [ ]:
sns.countplot( x='Persistency_Flag', hue='Race',data=data)

**Not Hispanic is dominant in Persistency_Flag**

In [ ]:
sns.countplot( x='Persistency_Flag', hue='Ethnicity', data=data)

**In all the regions the dominant was Not-Persistent**

In [ ]:
pd.crosstab(data['Region'], data['Persistency_Flag']).plot(kind='bar', figsize=(15, 6))
plt.legend(['Not Persistent', 'Persistent'])
plt.show()

**Female patients are more persistent of a drug than male**

In [ ]:
sns.histplot(x='Persistency_Flag', hue='Gender', data=data)

**The number of patients without having Dexa scan is higher**

In [ ]:
subplt1 = sns.countplot(x='Dexa_During_Rx', hue='Persistency_Flag', data=data)

**The ratio of the patients which are stable is much more higher than of the ratio of the improved patients**

In [ ]:
pd.crosstab(data["Gluco_Record_Prior_Ntm"], data["Gluco_Record_During_Rx"]).plot(kind='bar', figsize=(15, 6))
plt.show()

# Encoding

get all categorical columns


convert all categorical columns to numeric

In [ ]:
cat_columns = data.select_dtypes(['object']).columns

data[cat_columns] = data[cat_columns].apply(lambda x: pd.factorize(x)[0])


data

Check null values again

In [ ]:
data.isnull().sum()

In my opnion, we need to reduce some of features so these feature will be dropped and hence will have 50 column 

The columns Risk_Type_1_Insulin_Dependent_Diabetes, Risk_Osteogenesis_Imperfecta, Risk_Rheumatoid_Arthritis, Risk_Untreated_Chronic_Hyperthyroidism, Risk_Untreated_Chronic_Hypogonadism, Risk_Untreated_Early_Menopause, Risk_Patient_Parent_Fractured_Their_Hip ,Risk_Smoking_Tobacco, Risk_Chronic_Malnutrition_Or_Malabsorption, Risk_Chronic_Liver_Disease, Risk_Family_History_Of_Osteoporosis ,Risk_Low_Calcium_Intake, Risk_Vitamin_D_Insufficiency, Risk_Poor_Health_Frailty, Risk_Excessive_Thinness, Risk_Hysterectomy_Oophorectomy, Risk_Estrogen_Deficiency, Risk_Immobilization Risk_Recurring_Falls


In [ ]:
data = data.drop(['Risk_Type_1_Insulin_Dependent_Diabetes','Risk_Osteogenesis_Imperfecta','Risk_Rheumatoid_Arthritis',
'Risk_Untreated_Chronic_Hyperthyroidism','Risk_Untreated_Chronic_Hypogonadism','Risk_Untreated_Early_Menopause',
'Risk_Patient_Parent_Fractured_Their_Hip','Risk_Smoking_Tobacco','Risk_Chronic_Malnutrition_Or_Malabsorption',
'Risk_Chronic_Liver_Disease','Risk_Family_History_Of_Osteoporosis','Risk_Low_Calcium_Intake','Risk_Vitamin_D_Insufficiency',
'Risk_Poor_Health_Frailty','Risk_Excessive_Thinness','Risk_Hysterectomy_Oophorectomy','Risk_Estrogen_Deficiency'
,'Risk_Immobilization','Risk_Recurring_Falls'], axis = 1)



In [ ]:
data.columns

The Columns:
Comorb_Encounter_For_Screening_For_Malignant_Neoplasms,Comorb_Encounter_For_Immunization,Comorb_Encntr_For_General_Exam_W_O_Complaint,
_Susp_Or_Reprtd_Dx,Comorb_Vitamin_D_Deficiency,Comorb_Other_Joint_Disorder_Not_Elsewhere_Classified',
Comorb_Encntr_For_Oth_Sp_Exam_W_O_Complaint_Suspected_Or_Reprtd_Dx,Comorb_Long_Term_Current_Drug_Therapy, Comorb_Dorsalgia,Comorb_Personal_History_Of_Other_Diseases_And_Conditions,
Comorb_Other_Disorders_Of_Bone_Density_And_Structure,Comorb_Disorders_of_lipoprotein_metabolism_and_other_lipidemias,
Comorb_Osteoporosis_without_current_pathological_fracture,Comorb_Personal_history_of_malignant_neoplasm,Comorb_Gastro_esophageal_reflux_disease,
Concom_Cholesterol_And_Triglyceride_Regulating_Preparations,Concom_Narcotics, Concom_Systemic_Corticosteroids_Plain,
Concom_Anti_Depressants_And_Mood_Stabilisers,Concom_Fluoroquinolones, Concom_Cephalosporins,Concom_Macrolides_And_Similar_Types,Concom_Broad_Spectrum_Penicillins, Concom_Anaesthetics_General,Concom_Viral_Vaccines

Will be two columns have count of them one for columns that start with comorb and one for concom

In [ ]:
# Count_Of_Concomitancy
data['Count_Of_Concomitancy'] = data.iloc[:, 37:48].dot(np.ones(data.iloc[:, 37:48].shape[1]))

# Count_Of_Comorbidity
data['Count_Of_Comorbidity'] = data.iloc[:, 24:36].dot(np.ones(data.iloc[:, 24:36].shape[1]))




In [ ]:

data.head()

In [ ]:
data=data.drop(['Comorb_Encounter_For_Screening_For_Malignant_Neoplasms','Comorb_Encounter_For_Immunization',
'Comorb_Encntr_For_General_Exam_W_O_Complaint,_Susp_Or_Reprtd_Dx',
'Comorb_Vitamin_D_Deficiency','Comorb_Other_Joint_Disorder_Not_Elsewhere_Classified',
'Comorb_Encntr_For_Oth_Sp_Exam_W_O_Complaint_Suspected_Or_Reprtd_Dx','Comorb_Long_Term_Current_Drug_Therapy', 'Comorb_Dorsalgia',
'Comorb_Personal_History_Of_Other_Diseases_And_Conditions','Comorb_Other_Disorders_Of_Bone_Density_And_Structure',
'Comorb_Disorders_of_lipoprotein_metabolism_and_other_lipidemias','Comorb_Osteoporosis_without_current_pathological_fracture',
'Comorb_Personal_history_of_malignant_neoplasm','Comorb_Gastro_esophageal_reflux_disease','Concom_Cholesterol_And_Triglyceride_Regulating_Preparations',
'Concom_Narcotics', 'Concom_Systemic_Corticosteroids_Plain','Concom_Anti_Depressants_And_Mood_Stabilisers',
'Concom_Fluoroquinolones', 'Concom_Cephalosporins','Concom_Macrolides_And_Similar_Types',
'Concom_Broad_Spectrum_Penicillins', 'Concom_Anaesthetics_General','Concom_Viral_Vaccines'], axis = 1)



In [ ]:
data.head()

# **Models**
**Model Selection**

In [ ]:
y = data['Persistency_Flag']
y

In [ ]:
x = data.drop(['Persistency_Flag'],axis = 1)
x

In [ ]:
Y= y.values
X=x.values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)


# Logistic Regression

In [ ]:


lgr= LogisticRegression(solver="liblinear")

model =lgr.fit(x_train,y_train)
y_pred_lgr= model.predict(x_test)


In [ ]:

accuracy_score(y_test, y_pred_lgr)*100

In [ ]:

print(classification_report(y_test, y_pred_lgr))

# Decision Tree

In [ ]:
tree=DecisionTreeClassifier()
tree_model=tree.fit(x_train, y_train)
y_pred_tree=tree_model.predict(x_test)
accuracy_score(y_test, y_pred_tree)*100

In [ ]:
print(classification_report(y_test, y_pred_tree))

# Support Vector Machine

In [ ]:
svc=svm.SVC()
model_svc= svc.fit(x_train, y_train)
y_pred_svc= model_svc.predict(x_test)
accuracy_score(y_test, y_pred_svc)*100

In [ ]:
print(classification_report(y_test, y_pred_svc))

# Gradient Boosting Model

In [ ]:
gbm=GradientBoostingClassifier()
gbm_model=gbm.fit(x_train, y_train)
y_pred_gbm=gbm_model.predict(x_test)
accuracy_score(y_test, y_pred_gbm)*100


In [ ]:
print(classification_report(y_test, y_pred_gbm))

# Random Forest Classifier

In [ ]:

regressor =RandomForestClassifier()

#fit the model
model_rf =regressor.fit(x_train,y_train)
y_pred_rf= model.predict(x_test)
accuracy_score(y_test, y_pred_rf)*100

In [ ]:
print(classification_report(y_test, y_pred_rf))

# KNN 

In [ ]:
knn=KNeighborsClassifier()
knn_model=knn.fit(x_train, y_train)
y_pred= knn_model.predict(x_test)
accuracy_score(y_test, y_pred)*100


In [ ]:
print(classification_report(y_test, y_pred))

# Neural Network

In [ ]:
scaler=StandardScaler()
scaler.fit(x_train)
X_scaled=scaler.transform(x_train)
test_scaled=scaler.transform(x_test)
mlpc=MLPClassifier().fit(X_scaled, y_train)
y_pred_networks=mlpc.predict(test_scaled)
accuracy_score(y_test, y_pred_networks)*100

In [ ]:
print(classification_report(y_test, y_pred_networks))

# ROC-AUC

In [ ]:
y_score1 = model.predict(x_test)
y_score2 = model_rf.predict(x_test)
y_score3 = tree_model.predict(x_test)
y_score4 = model_svc.predict(x_test)
y_score5 = knn_model.predict(x_test)
y_score6 = gbm_model.predict(x_test)
y_score7 = mlpc.predict(test_scaled)


In [ ]:



fpr_lr, tpr_lr, threshold1 = roc_curve(y_test,  y_score1)
fpr_rfc, tpr_rfc, threshold1 =roc_curve(y_test,  y_score2)
fpr_dtc, tpr_dtc, threshold1 =roc_curve(y_test,  y_score3)
fpr_svc, tpr_svc, threshold1 = roc_curve(y_test,  y_score4)
fpr_knn, tpr_knn, threshold1 =roc_curve(y_test,  y_score5)
fpr_gbm, tpr_gbm, threshold1 =roc_curve(y_test,  y_score6)
fpr_mlpc, tpr_mlpc, threshold1 =roc_curve(y_test,  y_score7)



In [ ]:
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

In [ ]:
plt.subplots(1, figsize=(10,10))
plt.title('Receiver Operating Characteristic - All Models')
plt.plot(fpr_lr, tpr_lr, label = "Logistic Regression")
plt.plot(fpr_rfc, tpr_rfc, label = "Random Forest")
plt.plot(fpr_dtc, tpr_dtc, label = "Tree")
plt.plot(fpr_svc, tpr_svc,  label = "SVC")
plt.plot(fpr_knn, tpr_knn, label = "KNN")
plt.plot(fpr_gbm, tpr_gbm, label = "GBM")
plt.plot(fpr_mlpc, tpr_mlpc, label = "Neural Network")



plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive  Rate')
plt.xlabel('False Negative Rate')
plt.legend()
plt.show()

We split the dataset as train set and test set.

We have applied Logistic Regression, KNN, Random Forest Model, Decision Tree, Support Vector Machines, Gradient Boosting Model, and Neural Network Models.

we have calculated their accuracy scores and we obtained the following:

Gradient Boosting Model is the best fit model to our dataset with accuracy score 79.05.

We can apply Logistic Regression and Random Forest since their accuracy score 78